In [1]:
import torch
import torch.optim as optim
import pickle 

import numpy as np
import pandas as pd
from utils.vmf_batch import vMF

from models import SeqEncoder, SeqDecoder, Seq2Seq_VAE, PoolingClassifier, init_weights
from utils.training_utils import train, evaluate

## plotting ###

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
SEED = 17
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### load iterator

In [3]:
path = './data/%s/iterator/%s/%s_%i.pkl'
part = 'axon'
folder = 'M1_inh_data'
wl = 32

with open(path%(folder, part, 'train_iterator', wl), 'rb') as f:
    train_iterator = pickle.load(f)
    
with open(path%(folder, part, 'val_iterator', wl), 'rb') as f:
    val_iterator = pickle.load(f)    

In [4]:
src_data, trg_data, seq_len, indices, labels = list(train_iterator)[0]
bs, n_walks, walk_length, output_dim = src_data.shape

N_train = len(train_iterator.sampler.indices)
N_val = len(val_iterator.sampler.indices)

### load model

In [5]:
emb_dim = 32
latent_dim = 32
NUM_LAYERS = 2
dpout = .1
kap = 500
pool = 'max'
lr = 0.01
MASKING_ELEMENT = train_iterator.dataset.masking_el

# get number of labels, ignore -100 index
l = list(np.unique(labels))
if -100 in l:
    l.remove(-100)
NUM_CLASSES = len(l)

In [6]:
enc = SeqEncoder(output_dim, emb_dim, emb_dim, NUM_LAYERS, dpout)
dec = SeqDecoder(output_dim, emb_dim, emb_dim, NUM_LAYERS, dpout)
dist = vMF(latent_dim, kappa=kap)
model = Seq2Seq_VAE(enc, dec, dist, device).to(device)
classifier = PoolingClassifier(latent_dim, NUM_CLASSES, n_walks,dpout,pooling=pool).to(device)

KLD: 45.709938049316406


In [7]:
def calculate_loss(x, reconstructed_x, ignore_el=MASKING_ELEMENT):
    # reconstruction loss
    # x = [trg len, batch size * n walks, output dim]

    seq_len , bs, output_dim = x.shape
    mask = x[:,:,0] != ignore_el
    RCL = 0
    for d in range(output_dim):
        RCL += mse_loss(reconstructed_x[:,:,d][mask], x[:,:,d][mask])
    RCL /= output_dim
    
    return RCL

In [17]:
# load pre-trained model
# state_dict = torch.load('./models/5_populations/emb32_hid32_lat32_dp0.1_k500_max_frac1.0_scaled_sum_run1_best.pt')
state_dict = torch.load('./models/M1_inh/scratch/axon/scaled_vae_frac0.0_best_run2.pt')

In [13]:
torch.cuda.empty_cache()

In [18]:

N_EPOCHS= 200
save_path_model= './models/M1_inh/finetuned/%s/finetuned_scaled_vae_frac%.1f_best_run%i.pt'
save_path_losses = './models/M1_inh/finetuned/%s/finetuned_scaled_losses_frac%.1f_run%i.npy'

save_path_elapsed_time = './models/M1_inh/finetuned/%s/finetuned_scaled_elapsed_time_frac%.1f_run%i.npy'
# state_dict = torch.load('./models/5_populations/emb32_hid32_lat32_dp0.1_k500_max_frac1.0_scaled_sum_run1_best.pt')

start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

for frac in [1., .5, 0.]:#[1., .9, .5, .1]:
    
    runs = range(1,4)
        
    for run in runs:
        
    
        #state_dict = torch.load(save_path_model%(frac,run))
        
        #optimizer
        optimizer = optim.Adam(list(model.parameters()) + list(classifier.parameters()), lr=lr)
        # load pre-trained model
        if state_dict is not None:
            model.load_state_dict(state_dict['model_state_dict'])
            
#             losses = np.load('./models/5_populations/losses_emb32_hid32_lat32_dp0.1_k500_max_frac1.0_scaled_sum_1.npy')
#             losses = np.load(save_path_losses%(part, frac, run))
            losses = np.load("./models/M1_inh/scratch/%s/scaled_losses_frac0.0_run2.npy"%(part))
            training = list(losses[:,:2])
            validation = list(losses[:,2:])
           
            best_test_loss = losses[:,2].min()
            
        else:
            model.apply(init_weights)
            
            training = []
            validation=[]
        
        last_epoch = 0
        elapsed_time = np.zeros((N_EPOCHS))
        classifier.apply(init_weights)
        
        
        cross_entropy_loss = torch.nn.CrossEntropyLoss(reduction='sum', ignore_index=-100)
        mse_loss = torch.nn.MSELoss(reduction='sum')


        for e in range(last_epoch, N_EPOCHS):
            start.record()
            train_loss, train_class_loss = train(model, classifier, train_iterator, optimizer, 
                                               calculate_loss,cross_entropy_loss, clip=1, norm_p=None,
                                                 class_fraction=frac)
            val_loss, val_class_loss = evaluate(model,classifier, val_iterator,
                                                 calculate_loss, cross_entropy_loss, norm_p=None)

            train_loss /= N_train
            train_class_loss /= N_train
            val_loss /= N_val
            val_class_loss /=N_val
            
            end.record()
            #  Waits for everything to finish running
            torch.cuda.synchronize()
            elapsed_time[e] = start.elapsed_time(end) # milliseconds
            
            training += [[train_loss,train_class_loss]]
            validation += [[val_loss, val_class_loss]]
            print(f'Epoch {e}, Train Loss: {train_loss:.2f}, Val Loss: {val_loss:.2f}, Time elapsed [s]: {elapsed_time[e]/1000:.2f}')


            if e % 50 == 0 and e > 0:
                optimizer.param_groups[0]['lr'] = optimizer.param_groups[0]['lr']/2

            if best_test_loss > val_loss:
                best_test_loss = val_loss
                torch.save({'epoch': e,
                            'model_state_dict': model.state_dict(),
                            'optimizer_state_dict': optimizer.state_dict(),
                            'classifier_state_dict': classifier.state_dict()
                               },save_path_model%(part, frac, run))

                validation_ = np.array(validation)
                training_ = np.array(training)
                # [:,0] = training loss, [:,1] = training classification losss
                # [:,2] validation loss, [:,3] validation classification loss
                losses = np.hstack((training_, validation_))
                np.save(save_path_losses%(part,frac, run),losses)
                np.save(save_path_elapsed_time%(part,frac,run),elapsed_time)
        validation = np.array(validation)
        training = np.array(training)
        losses = np.hstack((training, validation))
        np.save(save_path_losses%(part, frac, run), losses)
        np.save(save_path_elapsed_time%(part,frac,run),elapsed_time)
        
torch.cuda.empty_cache()

Epoch 0, Train Loss: 1643.13, Val Loss: 2936.45, Time elapsed [s]: 14.79
Epoch 1, Train Loss: 738.55, Val Loss: 2130.07, Time elapsed [s]: 14.54
Epoch 2, Train Loss: 719.08, Val Loss: 1792.16, Time elapsed [s]: 14.53
Epoch 3, Train Loss: 625.12, Val Loss: 2188.79, Time elapsed [s]: 14.50
Epoch 4, Train Loss: 581.47, Val Loss: 4181.60, Time elapsed [s]: 14.48
Epoch 5, Train Loss: 502.78, Val Loss: 2856.79, Time elapsed [s]: 14.57
Epoch 6, Train Loss: 512.73, Val Loss: 4156.88, Time elapsed [s]: 14.57
Epoch 7, Train Loss: 546.67, Val Loss: 2235.42, Time elapsed [s]: 14.59
Epoch 8, Train Loss: 507.53, Val Loss: 1835.37, Time elapsed [s]: 14.54
Epoch 9, Train Loss: 497.46, Val Loss: 2733.17, Time elapsed [s]: 14.48
Epoch 10, Train Loss: 463.27, Val Loss: 2153.39, Time elapsed [s]: 14.52
Epoch 11, Train Loss: 453.04, Val Loss: 2291.29, Time elapsed [s]: 14.42
Epoch 12, Train Loss: 438.53, Val Loss: 2098.18, Time elapsed [s]: 14.44
Epoch 13, Train Loss: 420.88, Val Loss: 1842.25, Time elapse

Epoch 113, Train Loss: 258.22, Val Loss: 1398.22, Time elapsed [s]: 15.21
Epoch 114, Train Loss: 260.70, Val Loss: 1492.00, Time elapsed [s]: 14.75
Epoch 115, Train Loss: 254.63, Val Loss: 1361.90, Time elapsed [s]: 15.02
Epoch 116, Train Loss: 257.08, Val Loss: 1333.82, Time elapsed [s]: 14.67
Epoch 117, Train Loss: 261.62, Val Loss: 1489.85, Time elapsed [s]: 15.00
Epoch 118, Train Loss: 261.94, Val Loss: 1374.91, Time elapsed [s]: 14.63
Epoch 119, Train Loss: 259.25, Val Loss: 1328.03, Time elapsed [s]: 14.71
Epoch 120, Train Loss: 263.25, Val Loss: 1501.59, Time elapsed [s]: 14.63
Epoch 121, Train Loss: 254.49, Val Loss: 1340.35, Time elapsed [s]: 14.73
Epoch 122, Train Loss: 263.38, Val Loss: 1394.84, Time elapsed [s]: 14.68
Epoch 123, Train Loss: 253.26, Val Loss: 1442.17, Time elapsed [s]: 15.29
Epoch 124, Train Loss: 245.79, Val Loss: 1366.22, Time elapsed [s]: 15.35
Epoch 125, Train Loss: 248.14, Val Loss: 1398.50, Time elapsed [s]: 16.94
Epoch 126, Train Loss: 249.28, Val Los

Epoch 25, Train Loss: 414.53, Val Loss: 1681.82, Time elapsed [s]: 20.02
Epoch 26, Train Loss: 393.46, Val Loss: 1685.07, Time elapsed [s]: 20.01
Epoch 27, Train Loss: 400.64, Val Loss: 1618.61, Time elapsed [s]: 20.07
Epoch 28, Train Loss: 374.41, Val Loss: 1935.56, Time elapsed [s]: 20.03
Epoch 29, Train Loss: 385.00, Val Loss: 1625.30, Time elapsed [s]: 20.41
Epoch 30, Train Loss: 395.07, Val Loss: 1629.97, Time elapsed [s]: 20.16
Epoch 31, Train Loss: 374.83, Val Loss: 1799.76, Time elapsed [s]: 20.14
Epoch 32, Train Loss: 395.92, Val Loss: 1489.17, Time elapsed [s]: 20.08
Epoch 33, Train Loss: 388.59, Val Loss: 1555.34, Time elapsed [s]: 19.99
Epoch 34, Train Loss: 391.16, Val Loss: 1538.39, Time elapsed [s]: 20.09
Epoch 35, Train Loss: 376.90, Val Loss: 1528.09, Time elapsed [s]: 19.95
Epoch 36, Train Loss: 386.64, Val Loss: 1467.23, Time elapsed [s]: 20.04
Epoch 37, Train Loss: 391.52, Val Loss: 1504.39, Time elapsed [s]: 20.16
Epoch 38, Train Loss: 374.97, Val Loss: 1534.49, Ti

Epoch 137, Train Loss: 251.94, Val Loss: 1581.37, Time elapsed [s]: 20.09
Epoch 138, Train Loss: 250.25, Val Loss: 1502.02, Time elapsed [s]: 20.01
Epoch 139, Train Loss: 242.03, Val Loss: 1593.59, Time elapsed [s]: 20.07
Epoch 140, Train Loss: 249.58, Val Loss: 1562.06, Time elapsed [s]: 20.03
Epoch 141, Train Loss: 243.26, Val Loss: 1581.38, Time elapsed [s]: 19.98
Epoch 142, Train Loss: 248.23, Val Loss: 1705.51, Time elapsed [s]: 20.18
Epoch 143, Train Loss: 253.72, Val Loss: 1528.41, Time elapsed [s]: 19.98
Epoch 144, Train Loss: 247.32, Val Loss: 1562.89, Time elapsed [s]: 20.09
Epoch 145, Train Loss: 245.71, Val Loss: 1631.18, Time elapsed [s]: 20.02
Epoch 146, Train Loss: 240.30, Val Loss: 1575.01, Time elapsed [s]: 20.05
Epoch 147, Train Loss: 242.22, Val Loss: 1570.78, Time elapsed [s]: 20.06
Epoch 148, Train Loss: 243.24, Val Loss: 1563.88, Time elapsed [s]: 20.02
Epoch 149, Train Loss: 243.00, Val Loss: 1662.93, Time elapsed [s]: 19.98
Epoch 150, Train Loss: 245.05, Val Los

Epoch 49, Train Loss: 380.85, Val Loss: 2207.63, Time elapsed [s]: 20.13
Epoch 50, Train Loss: 401.71, Val Loss: 1863.71, Time elapsed [s]: 20.07
Epoch 51, Train Loss: 351.86, Val Loss: 1524.21, Time elapsed [s]: 20.02
Epoch 52, Train Loss: 362.52, Val Loss: 1508.11, Time elapsed [s]: 20.11
Epoch 53, Train Loss: 358.06, Val Loss: 1654.88, Time elapsed [s]: 20.02
Epoch 54, Train Loss: 355.77, Val Loss: 1501.37, Time elapsed [s]: 20.08
Epoch 55, Train Loss: 351.58, Val Loss: 1532.94, Time elapsed [s]: 19.96
Epoch 56, Train Loss: 354.72, Val Loss: 1729.52, Time elapsed [s]: 20.02
Epoch 57, Train Loss: 357.80, Val Loss: 1548.31, Time elapsed [s]: 20.06
Epoch 58, Train Loss: 336.92, Val Loss: 1490.01, Time elapsed [s]: 15.32
Epoch 59, Train Loss: 346.45, Val Loss: 1672.68, Time elapsed [s]: 14.56
Epoch 60, Train Loss: 351.50, Val Loss: 1593.87, Time elapsed [s]: 14.57
Epoch 61, Train Loss: 338.09, Val Loss: 1520.72, Time elapsed [s]: 14.51
Epoch 62, Train Loss: 349.59, Val Loss: 1571.18, Ti

...

Epoch 40, Train Loss: 284.68, Val Loss: 2031.36, Time elapsed [s]: 20.07
Epoch 41, Train Loss: 298.50, Val Loss: 2528.29, Time elapsed [s]: 20.04
Epoch 42, Train Loss: 279.86, Val Loss: 1626.96, Time elapsed [s]: 20.07
Epoch 43, Train Loss: 272.66, Val Loss: 1960.67, Time elapsed [s]: 19.99
Epoch 44, Train Loss: 281.86, Val Loss: 1702.80, Time elapsed [s]: 20.14
Epoch 45, Train Loss: 277.59, Val Loss: 1710.36, Time elapsed [s]: 20.09
Epoch 46, Train Loss: 278.07, Val Loss: 1882.25, Time elapsed [s]: 20.12
Epoch 47, Train Loss: 275.90, Val Loss: 1723.17, Time elapsed [s]: 20.12
Epoch 48, Train Loss: 283.94, Val Loss: 1638.19, Time elapsed [s]: 20.19
Epoch 49, Train Loss: 289.48, Val Loss: 1997.29, Time elapsed [s]: 20.08
Epoch 50, Train Loss: 288.11, Val Loss: 1663.70, Time elapsed [s]: 20.06
Epoch 51, Train Loss: 274.33, Val Loss: 1637.02, Time elapsed [s]: 20.33
Epoch 52, Train Loss: 272.62, Val Loss: 1575.28, Time elapsed [s]: 20.03
Epoch 53, Train Loss: 273.67, Val Loss: 1824.33, Ti

Epoch 152, Train Loss: 253.48, Val Loss: 1527.39, Time elapsed [s]: 20.03
Epoch 153, Train Loss: 264.43, Val Loss: 1524.94, Time elapsed [s]: 20.00
Epoch 154, Train Loss: 257.05, Val Loss: 1584.61, Time elapsed [s]: 20.03
Epoch 155, Train Loss: 252.97, Val Loss: 1613.02, Time elapsed [s]: 20.07
Epoch 156, Train Loss: 259.24, Val Loss: 1559.38, Time elapsed [s]: 19.95
Epoch 157, Train Loss: 253.97, Val Loss: 1515.95, Time elapsed [s]: 20.08
Epoch 158, Train Loss: 263.44, Val Loss: 1514.70, Time elapsed [s]: 20.01
Epoch 159, Train Loss: 258.30, Val Loss: 1598.38, Time elapsed [s]: 20.08
Epoch 160, Train Loss: 267.40, Val Loss: 1522.50, Time elapsed [s]: 19.99
Epoch 161, Train Loss: 258.31, Val Loss: 1502.35, Time elapsed [s]: 20.00
Epoch 162, Train Loss: 259.49, Val Loss: 1515.13, Time elapsed [s]: 20.04
Epoch 163, Train Loss: 259.31, Val Loss: 1518.51, Time elapsed [s]: 19.95
Epoch 164, Train Loss: 262.50, Val Loss: 1506.75, Time elapsed [s]: 19.93
Epoch 165, Train Loss: 261.36, Val Los

Epoch 64, Train Loss: 277.36, Val Loss: 1908.68, Time elapsed [s]: 14.51
Epoch 65, Train Loss: 274.62, Val Loss: 1514.12, Time elapsed [s]: 14.54
Epoch 66, Train Loss: 260.08, Val Loss: 1480.38, Time elapsed [s]: 14.58
Epoch 67, Train Loss: 271.42, Val Loss: 1528.64, Time elapsed [s]: 14.57
Epoch 68, Train Loss: 267.62, Val Loss: 1520.68, Time elapsed [s]: 14.57
Epoch 69, Train Loss: 260.85, Val Loss: 1527.89, Time elapsed [s]: 14.54
Epoch 70, Train Loss: 268.42, Val Loss: 1520.92, Time elapsed [s]: 14.54
Epoch 71, Train Loss: 267.25, Val Loss: 1644.07, Time elapsed [s]: 14.61
Epoch 72, Train Loss: 258.31, Val Loss: 1573.40, Time elapsed [s]: 14.59
Epoch 73, Train Loss: 275.82, Val Loss: 1615.47, Time elapsed [s]: 14.63
Epoch 74, Train Loss: 265.50, Val Loss: 2166.67, Time elapsed [s]: 14.55
Epoch 75, Train Loss: 272.72, Val Loss: 1540.94, Time elapsed [s]: 14.58
Epoch 76, Train Loss: 270.04, Val Loss: 1438.90, Time elapsed [s]: 14.50
Epoch 77, Train Loss: 274.76, Val Loss: 1591.88, Ti

Epoch 176, Train Loss: 172.38, Val Loss: 1617.75, Time elapsed [s]: 14.44
Epoch 177, Train Loss: 179.37, Val Loss: 1483.16, Time elapsed [s]: 14.55
Epoch 178, Train Loss: 186.35, Val Loss: 1496.56, Time elapsed [s]: 14.53
Epoch 179, Train Loss: 185.84, Val Loss: 1654.20, Time elapsed [s]: 14.52
Epoch 180, Train Loss: 182.97, Val Loss: 1542.34, Time elapsed [s]: 14.51
Epoch 181, Train Loss: 172.61, Val Loss: 1442.02, Time elapsed [s]: 14.60
Epoch 182, Train Loss: 173.78, Val Loss: 1495.42, Time elapsed [s]: 14.54
Epoch 183, Train Loss: 176.56, Val Loss: 1660.81, Time elapsed [s]: 14.55
Epoch 184, Train Loss: 173.72, Val Loss: 1501.48, Time elapsed [s]: 14.58
Epoch 185, Train Loss: 173.46, Val Loss: 1519.80, Time elapsed [s]: 14.59
Epoch 186, Train Loss: 182.56, Val Loss: 1742.14, Time elapsed [s]: 14.55
Epoch 187, Train Loss: 183.16, Val Loss: 1549.66, Time elapsed [s]: 14.54
Epoch 188, Train Loss: 170.22, Val Loss: 1490.30, Time elapsed [s]: 14.59
Epoch 189, Train Loss: 171.64, Val Los

Epoch 89, Train Loss: 260.88, Val Loss: 1843.15, Time elapsed [s]: 14.48
Epoch 90, Train Loss: 223.31, Val Loss: 1441.19, Time elapsed [s]: 14.50
Epoch 91, Train Loss: 238.24, Val Loss: 1520.29, Time elapsed [s]: 14.45
Epoch 92, Train Loss: 250.64, Val Loss: 1945.31, Time elapsed [s]: 14.43
Epoch 93, Train Loss: 217.34, Val Loss: 1480.28, Time elapsed [s]: 14.51
Epoch 94, Train Loss: 231.71, Val Loss: 1457.70, Time elapsed [s]: 14.51
Epoch 95, Train Loss: 223.67, Val Loss: 1694.67, Time elapsed [s]: 14.45
Epoch 96, Train Loss: 217.81, Val Loss: 1493.29, Time elapsed [s]: 14.50
Epoch 97, Train Loss: 247.89, Val Loss: 1775.00, Time elapsed [s]: 14.47
Epoch 98, Train Loss: 255.13, Val Loss: 2098.56, Time elapsed [s]: 14.50
Epoch 99, Train Loss: 236.82, Val Loss: 1664.59, Time elapsed [s]: 14.73
Epoch 100, Train Loss: 211.87, Val Loss: 1405.64, Time elapsed [s]: 14.55
Epoch 101, Train Loss: 211.03, Val Loss: 1501.51, Time elapsed [s]: 14.55
Epoch 102, Train Loss: 212.47, Val Loss: 1663.58,